In [1]:
# Importing Libraries required for NLP

import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Importing Libraries needed for Tensorflow processing

import tensorflow.compat.v1 as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JoãoCabral\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
#!cd

In [3]:
# importing .json file for training the model.

with open("C:/Users/JoãoCabral/JC88Main/final_project/Final-Project/intents/intents.json") as json_data: 
    intents = json.load(json_data)
    # Loading data from intents.json file to have variables

## TOKENIZE

In [4]:
# Creating empty lists for appending the data after processing NLP
words = []
documents = []
classes = []

# This list will be used for ignoring all unwanted punctuation marks.
ignore = ["?"]

# Running a loop through each "intents" in intents.json > ["patterns"]
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        # tokenizing each and every word in the sentence by using word tokenizer and storing in w
        w = nltk.word_tokenize(pattern) 
        
        # Adding tokenized words to words = [] empty list that was created previously
        words.extend(w) 
        #print(words)
        
        # Adding words to documents = [] empty list with "tag" on intents.json > ["tag"]
        documents.append((w, intent["tag"]))
        #print(documents)
        
        # Adding only "tag" to our classes = [] list
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])  #If tag is not present in classes[] list, it will append into it.
            #print(classes)
            

## STEMMING

In [5]:
# Performing Stemming by using stemmer.stem() and lower each word 
# Running loop in words[] list and ignoring punctuation marks present in ignore[]

words = [stemmer.stem(w.lower()) for w in words if w not in ignore]  
words = sorted(list(set(words)))  #Removing Duplicates in words[]

# Removing Duplicate Classes
classes = sorted(list(set(classes)))

# Printing length of lists we formed
print(len(documents),"Documents \n")
print(len(classes),"Classes \n")
print(len(words), "Stemmed Words ")

121 Documents 

37 Classes 

129 Stemmed Words 


## BAG OF WORDS

In [6]:
# Creating Training Data and output which will be further used for training
training = []
output = []

# Creating empty array for output
output_empty = [0] * len(classes)

# Creating Training set and bag of words for each sentence
for doc in documents:
    bag = [] # Creating empty bag of words
    
    # Storing list of tokenized words for the documents[]
    pattern_words = doc[0]
    #print(pattern_words)
    
    # Again Stemming each word from pattern_words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    #print(pattern_words)
    
    # Creating "full" bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # Display output for selected "tag" - 1 for selected tag and 0 for other tags (see below)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])

In [7]:
random.shuffle(training) # Shuffling the data
training = np.array(training) # Converting training data into numpy array for better performance

# Creating Training Lists (x-y split) from bag of words for modeling
train_x = list(training[:,0]) # Feature
train_y = list(training[:,1]) # Label

C:\Users\JoãoCabral\AppData\Local\Temp\ipykernel_26028\56795269.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training) # Converting training data into numpy array for better performance


Now that we have our data half structured we can go to the next phase.

## NLP, Model Definition, Model Train, Model fit and save

In [8]:
tf.reset_default_graph() # Reset Underlying Graph Data with Tensorflow

# Building a Neural Network - 2 layer NN that relates data with human behaviour (system learning)
net = tflearn.input_data(shape=[None, len(train_x[0])]) # Data Feeding layer
net = tflearn.fully_connected(net, 10) # Hidden Layer1 - 10 nodes
net = tflearn.fully_connected(net, 10) # Hidden Layer2 - 10 nodes
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax") # Softmax - The softmax activation transforms the raw outputs of the neural network into a vector of probabilities.
net = tflearn.regression(net) # Regression Layer

# Defining Model (tensorboard from Tensorflow) - (DNN stands for Deep Neural Network)
model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 
# "tflearn_logs" = Tensorboard parameter that enables logging

# Now that model is setup it need to be trainned
# n_epoch is the number of times that model will se our data during training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True) # Combination of x_y and Tf model
model.save("model.tflearn") # Save model not to be trainned again

# Every time the intents.json file is updated, I have to train the model

Training Step: 15999  | total loss: 0.12497 | time: 0.056s
| Adam | epoch: 1000 | loss: 0.12497 - acc: 0.9317 -- iter: 120/121
Training Step: 16000  | total loss: 0.13479 | time: 0.060s
| Adam | epoch: 1000 | loss: 0.13479 - acc: 0.9261 -- iter: 121/121
--
INFO:tensorflow:C:\Users\JoãoCabral\JC88Main\final_project\Final-Project\notebooks\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\JoãoCabral\JC88Main\final_project\Final-Project\notebooks\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\JoãoCabral\JC88Main\final_project\Final-Project\notebooks\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\JoãoCabral\JC88Main\final_project\Final-Project\notebooks\model.tflearn.meta
INFO:tensorflow:100


until here, data has gone through NLP two times 1-nltk / 2-tflearn (tensorflow) 

In [9]:
# Importing pickle module
import pickle

# Save data structures
pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [10]:
# Restoring all data structure
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [11]:
with open("C:/Users/JoãoCabral/JC88Main/final_project/Final-Project/intents/intents.json") as json_data:
    intents = json.load(json_data)  # Loading our json_data

In [12]:
# Loading the saved model
model.load("./model.tflearn")

INFO:tensorflow:Restoring parameters from C:\Users\JoãoCabral\JC88Main\final_project\Final-Project\notebooks\model.tflearn


## Handling user input

In [13]:
# Cleaning User Input with function
def clean_up_sentence(sentence):
    
    # Tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    
    # Stemming each word from the user's input
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

# Returning bag of words array: 0 or 1 or each word in the bag that exists in as we have declared in above lines
def bow(sentence, words, show_details=False):
        
    sentence_words = clean_up_sentence(sentence)
    
    # Generating bag of words from the sentence that user entered
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

## Adding Context

In [14]:
# Adding some context to the conversation for better results - Therefore CONTEXTUAL CHATBOT

# Create a dictionary to hold user's context
context = {}

ERROR_THRESHOLD = 0.25 # Number of intents to be filtered and stored in a threshold
def classify(sentence):
    
    # Generating probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    
    # Filter out predictions from threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    # Sorting by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    
    # Return tuple of intent + probability
    return return_list

def response(sentence, userID='123', show_details=False): # Response function
    results = classify(sentence)
    
    # If we have a classification then find the matching intent tag
    if results:
        
        # Loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                
                # Find a tag matching the first result
                if i['tag'] == results[0][0]:
                    
                    # Set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # Check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        # A random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [15]:
answer = True
while answer:
    question=input("Please enter your question: ")
    response(question)
    print()
    print("Do you have more questions? {1-> Yes / 2-> No}")
    user_answer = int(input("Enter your choice: "))
    while user_answer not in [1,2]:
        print("Answer not valid, please select 1 for 'Yes' or 2 for 'No':")
        user_answer = int(input("Enter your choice: "))
    if ( user_answer == 1 ):
        answer=True
    else:
        answer=False

Please enter your question: 2
Movie shooting location was in Oxbow Regional Park in Oregon and in Warner Bros. Studios in California.

Do you have more questions? {1-> Yes / 2-> No}
Enter your choice: 2


In [22]:
from tkinter import *

root = Tk()

root.title('Cinebot')

root.geometry('400x500')

main_menu = Menu(root)

file_menu = Menu(root)
main_menu.add_command(label='New')
file_menu.add_command(label='Save As..')
file_menu.add_command(label='Exit')



main_menu.add_cascade(label='File', menu = file_menu)
main_menu.add_command(label='Edit')
main_menu.add_command(label='Quit')
root.config(menu=main_menu)


chatWindow = Text(root, bd=1, bg='black', width = 50, height = 8)
chatWindow.place(x=6, y=6, height= 385, width = 370)


messageWindow = Text(root, bg='black', width = 30, height = 4)
messageWindow.place(x=128, y=400, height = 88, width = 260)


Button = Button(root, text='Send', bg='light blue', activebackground='blue', width = 12, height = 5, font=('Arial', 18))
Button.place(x=6, y=400, height = 88, width = 120)


scrollbar = Scrollbar(root, command=chatWindow.yview())
scrollbar.place(x=375, y=5, height = 385)

root.mainloop()

After response you can check context of dialougue by typing 'context' on the cell

In [16]:
context

{}

In [17]:
response("you are useless!", show_details=True)

tag: Rejection
One day... I will summon Skynet
